In [1]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:58818 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 17.18 GB


In [2]:
import pandas as pd
df = pd.read_csv("zappos.csv")
assert (df.columns == ["worker_id", "head", "b", "c"]).all()
df = df[["head", "b", "c"]]
df.columns = ["head", "winner", "loser"]
df = df.astype("int8")
print(df.dtypes)
df.head()

head      int8
winner    int8
loser     int8
dtype: object


,head,winner,loser
0,13,47,53
1,59,12,6
2,7,78,55
3,13,47,53
4,59,6,12


In [3]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, random_state=42, test_size=0.2)

In [4]:
train.dtypes

head      int8
winner    int8
loser     int8
dtype: object

In [5]:
def _get_query_freq(h, n=85, df=None):
    queries = [
        (h, o1, o2)
        for o1, o2 in itertools.product(targets, targets)
        if h != o1 and h != o2 and o1 != o2
    ]
    freqs = {}
    key = lambda h, o1, o2: f"{h}-{o1}-{o2}"
    for h, o1, o2 in queries:
        idx = (df["head"] == h)
        idx &= (df["winner"] == o1) | (df["winner"] == o2)
        idx &= (df["loser"] == o1) | (df["loser"] == o2)
        relevant = df[idx]
        o1wins = (relevant["winner"] == o1).sum()
        o2wins = (relevant["winner"] == o2).sum()
        freqs[key(h, o1, o2)] = (int(o1wins), int(o2wins))
        freqs[key(h, o2, o1)] = (int(o2wins), int(o1wins))
    return freqs

In [6]:
import itertools
N = df.values.max()
targets = list(range(N + 1))
futures = client.map(_get_query_freq, targets, n=N, df=train)
winner_freqs = client.gather(futures)

In [7]:
len(winner_freqs)

85

In [8]:
for i, winner1 in enumerate(winner_freqs):
    for j, winner2 in enumerate(winner_freqs):
        if i == j:
            continue
        key1 = set(winner1.keys())
        key2 = set(winner2.keys())
        assert key1.intersection(key2) == set()
        assert len(key1.union(key2)) == len(key1) + len(key2)

In [9]:
from toolz.dicttoolz import merge
total = merge(*winner_freqs)
len(total)

592620

In [17]:
import msgpack
with open("freqs.msgpack", "wb") as f:
    msgpack.dump(total, f)

In [19]:
with open("freqs.msgpack", "rb") as f:
    data = msgpack.load(f)

In [22]:
!ls

Generate-freqs-msgpack.ipynb freqs.msgpack.zip
__MACOSX                     zappos.csv
dask-worker-space            zappos.csv.zip
freqs.msgpack


In [24]:
!rm freqs.msgpack.zip

In [25]:
!zip freqs.msgpack.zip freqs.msgpack

  adding: freqs.msgpack (deflated 79%)
